# Predict Future Sale - Baseline
## Final project for "How to win a data science competition" Coursera course
https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data  
>Student: Rafael Caneiro de Oliveira  
>Email: rafael.caneiro@gmail.com  
>Date: 04/08/2020

## Load the data

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path

PATH = Path.cwd().parent
DATA_PATH = Path(PATH, "./data/raw/") 

In [3]:
import h2o

<ipython-input-3-24b3e465bd21>:2: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


H2O session _sid_b7c9 closed.


In [12]:
sales_train_df = pd.read_csv(Path(DATA_PATH,"sales_train.csv"))
test_df = pd.read_csv(Path(DATA_PATH,"test.csv"))
items_df = pd.read_csv(Path(DATA_PATH,"items.csv"))

train_df = pd.merge(sales_train_df,
                    items_df[["item_id", "item_category_id"]],
                    how="inner",
                    on="item_id")

train_df["date"] = pd.to_datetime(train_df["date"], format="%d.%m.%Y")

test_df = pd.merge(test_df,
                   items_df[["item_id", "item_category_id"]],
                   how="inner",
                   on="item_id")

test_df["date_block_num"] = 34

In [13]:
print(train_df.shape)
train_df.head()

(2935849, 7)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,2013-01-02,0,59,22154,999.0,1.0,37
1,2013-01-23,0,24,22154,999.0,1.0,37
2,2013-01-20,0,27,22154,999.0,1.0,37
3,2013-01-02,0,25,22154,999.0,1.0,37
4,2013-01-03,0,25,22154,999.0,1.0,37


In [14]:
print(test_df.shape)
test_df.head()

(214200, 5)


,ID,shop_id,item_id,item_category_id,date_block_num
0,0,5,5037,19,34
1,5100,4,5037,19,34
2,10200,6,5037,19,34
3,15300,3,5037,19,34
4,20400,2,5037,19,34


## Basic Stats Reminder

In [15]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
date_block_num,2935849.0,14.569911,9.422988,0.0,7.0,14.0,23.0,33.0
shop_id,2935849.0,33.001728,16.226973,0.0,22.0,31.0,47.0,59.0
item_id,2935849.0,10197.227057,6324.297354,0.0,4476.0,9343.0,15684.0,22169.0
item_price,2935849.0,890.853233,1729.799631,-1.0,249.0,399.0,999.0,307980.0
item_cnt_day,2935849.0,1.242641,2.618834,-22.0,1.0,1.0,1.0,2169.0
item_category_id,2935849.0,40.001383,17.100759,0.0,28.0,40.0,55.0,83.0


- investigar valor -1 em `item_price`
- nlp com dados dos nomes dos produtos e categorias >> Russion Word2Vec

## Outliers
https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

In [16]:
cols = ["item_price", "item_cnt_day"]
for col in cols:
    upperbound = np.percentile(train_df[col], 99)
    train_df.loc[train_df[col] > upperbound, "is_outlier"] = 1
    
train_df.is_outlier.fillna(0, inplace=True)

train_df.is_outlier.value_counts()

train_df = train_df[train_df.is_outlier==0]

## Model

In [17]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42,
                           n_estimators=500,
                           verbose=2,
                           n_jobs=-1)

features = ["date_block_num", "shop_id",
            "item_id", "item_category_id"]

X_train = train_df[features]
y_train = train_df["item_cnt_day"]

mdl = rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


building tree 1 of 500building tree 2 of 500

building tree 3 of 500
building tree 4 of 500building tree 5 of 500

building tree 6 of 500
building tree 7 of 500building tree 8 of 500
building tree 9 of 500building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500

building tree 14 of 500

building tree 15 of 500
building tree 16 of 500
building tree 17 of 500building tree 18 of 500

building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500building tree 24 of 500
building tree 25 of 500

building tree 26 of 500


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   13.4s


building tree 27 of 500building tree 28 of 500building tree 29 of 500


building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500building tree 37 of 500

building tree 38 of 500building tree 39 of 500

building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500building tree 47 of 500

building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500building tree 55 of 500

building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68

[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  1.9min


building tree 146 of 500
building tree 147 of 500building tree 148 of 500

building tree 149 of 500
building tree 150 of 500
building tree 151 of 500building tree 152 of 500building tree 153 of 500


building tree 154 of 500
building tree 155 of 500building tree 156 of 500

building tree 157 of 500
building tree 158 of 500
building tree 159 of 500
building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500
building tree 165 of 500
building tree 166 of 500
building tree 167 of 500
building tree 168 of 500
building tree 169 of 500
building tree 170 of 500building tree 171 of 500
building tree 172 of 500

building tree 173 of 500building tree 174 of 500building tree 175 of 500


building tree 176 of 500
building tree 177 of 500
building tree 178 of 500building tree 179 of 500

building tree 180 of 500
building tree 181 of 500
building tree 182 of 500building tree 183 of 500

building tree 184 of 500
building tree 185 of 500


[Parallel(n_jobs=-1)]: Done 333 tasks      | elapsed:  4.4min


building tree 349 of 500
building tree 350 of 500
building tree 351 of 500
building tree 352 of 500
building tree 353 of 500
building tree 354 of 500
building tree 355 of 500
building tree 356 of 500
building tree 357 of 500
building tree 358 of 500
building tree 359 of 500
building tree 360 of 500
building tree 361 of 500
building tree 362 of 500
building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500building tree 373 of 500

building tree 374 of 500
building tree 375 of 500
building tree 376 of 500
building tree 377 of 500
building tree 378 of 500
building tree 379 of 500building tree 380 of 500

building tree 381 of 500
building tree 382 of 500
building tree 383 of 500
building tree 384 of 500
building tree 385 of 500
building tree 386 of 500
building tree 387 of 500
building tree 388 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  6.8min finished


## Evaluation

In [18]:
predictions = mdl.predict(test_df[features])

predictions_s = pd.Series(np.round(predictions,0), name="item_cnt_month")

submission = pd.concat([test_df, predictions_s],
                       axis=1)

submission["item_cnt_month"].clip(0,20, inplace=True)

submission.head()

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    1.0s
[Parallel(n_jobs=16)]: Done 333 tasks      | elapsed:    2.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    3.0s finished


,ID,shop_id,item_id,item_category_id,date_block_num,item_cnt_month
0,0,5,5037,19,34,1.0
1,5100,4,5037,19,34,1.0
2,10200,6,5037,19,34,1.0
3,15300,3,5037,19,34,1.0
4,20400,2,5037,19,34,1.0


In [19]:
submission[["ID", "item_cnt_month"]].to_csv("submission.csv", index=False)

In [20]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission.csv -m "Baseline3"

100%|███████████████████████████████████████| 2.14M/2.14M [00:02<00:00, 904kB/s]
Successfully submitted to Predict Future Sales